In [1]:
import yfinance as yf
import requests
import time
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import pandas as pd
from datetime import datetime

import numpy as np

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from transformers import BertTokenizer
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from datetime import datetime

from transformers import (
    BertTokenizer, 
    BertForSequenceClassification, 
    
)
import evaluate
import matplotlib.pyplot as plt
import seaborn as sns
import os

from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

/Users/jiunyih/venv/p311_dsa4265/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define the search parameters for cryptocurrency-related filings
url = "https://www.sec.gov/cgi-bin/browse-edgar"
params = {
    "company": "Grayscale Bitcoin Cash Trust",  # Searching for Grayscale Bitcoin Cash Trust
    "match": "starts-with",
    "filenum": "",
    "State": "",
    "Country": "",
    "SIC": "",
    "myowner": "exclude",
    "action": "getcompany",
    "type": "10-K", # 10-Q for quarter report    
}

# Add a User-Agent header to comply with SEC's guidelines
headers = {
    "User-Agent": "YourAppName/1.0 (your-email@example.com)"
}

# Send the request to the SEC EDGAR API
response = requests.get(url, headers=headers, params=params)

# Parse the HTML response
soup = BeautifulSoup(response.text, "html.parser")

# Extract the links or filings
filing_links = []
for link in soup.find_all("a", href=True):
    # Only add links that end with "-index.htm"
    if link['href'].endswith("-index.htm"):
        filing_links.append(link['href'])

all_filling_links = []
# Print the filtered links
for filing in filing_links:
    all_filling_links.append(f"https://www.sec.gov{filing}")
    print(f"Filing link: https://www.sec.gov{filing}")

# Add a delay between requests to avoid rate limit issues
time.sleep(1)

Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017024104235/0000950170-24-104235-index.htm
Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017023045923/0000950170-23-045923-index.htm
Filing link: https://www.sec.gov/Archives/edgar/data/1732409/000095017022018010/0000950170-22-018010-index.htm


In [3]:
docs_dic = {}

for link in all_filling_links:
    try:
        # Generate document link
        doc = link.replace("-index", "").replace("htm", "txt")
        print(f"🔗 Generated Document Link: {doc}")

        # Send the GET request
        response = requests.get(link, headers=headers)
        response.raise_for_status()  # Check for request errors

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the "Period of Report"
        period_of_report = None
        for div in soup.find_all('div', class_='formGrouping'):
            if "Period of Report" in div.text:
                period_of_report = div.find('div', class_='info').text.strip()

        # Format the period_of_report
        formatted_period_of_report = period_of_report.replace('-', '') if period_of_report else "N/A"
        print(f"✅ Formatted Period of Report: {formatted_period_of_report}")

        # Extract necessary parts from the link
        parts = link.split('/')
        cik = parts[6]  
        filing_identifier = parts[7]  

        # Construct the new URL
        new_link = f"https://www.sec.gov/ix?doc=/Archives/edgar/data/{cik}/{filing_identifier}/bchg-{formatted_period_of_report}.htm"
        print(f"🔗 Generated Report Link: {new_link}\n")

        # Store in the dictionary
        docs_dic[period_of_report] = {
            "period_of_report": formatted_period_of_report,
            "report_link": new_link,
            "doc_link": doc
        }

    except Exception as e:
        print(f"⚠️ Error processing {link}: {e}")

# Print the final dictionary
print(docs_dic)

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017024104235/0000950170-24-104235.txt
✅ Formatted Period of Report: 20240630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017024104235/bchg-20240630.htm

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017023045923/0000950170-23-045923.txt
✅ Formatted Period of Report: 20230630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017023045923/bchg-20230630.htm

🔗 Generated Document Link: https://www.sec.gov/Archives/edgar/data/1732409/000095017022018010/0000950170-22-018010.txt
✅ Formatted Period of Report: 20220630
🔗 Generated Report Link: https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017022018010/bchg-20220630.htm

{'2024-06-30': {'period_of_report': '20240630', 'report_link': 'https://www.sec.gov/ix?doc=/Archives/edgar/data/1732409/000095017024104235/bchg-20240630.htm', 'doc_link'

In [ ]:
from sec_edgar_downloader import Downloader

# Provide your email address (as required by SEC)
dl = Downloader(email_address="your_email@example.com", download_folder="./sec_data", company_name="school")

dl.get(
    "10-K",
    ticker_or_cik = "AAPL",
    after = "2015-01-01"
)

dl.get(
    "10-K",
    ticker_or_cik = "MSFT",
    after = "2015-01-01"
)


10

In [3]:
import requests
import os
import time

CIK = "0000320193"  # Apple
EMAIL = "your_email@example.com"  # Use your real email
SAVE_FOLDER = "apple_10k_txts"
os.makedirs(SAVE_FOLDER, exist_ok=True)

HEADERS = {
    "User-Agent": f"MyApp/1.0 ({EMAIL})"
}

def get_10k_filing_info(cik):
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    res = requests.get(url, headers=HEADERS)
    data = res.json()
    
    info = []
    for i, form in enumerate(data["filings"]["recent"]["form"]):
        if form == "10-K":
            acc_num = data["filings"]["recent"]["accessionNumber"][i]  # e.g., 0000320193-23-000106
            acc_num_clean = acc_num.replace("-", "")  # e.g., 000032019323000106
            info.append((acc_num_clean, acc_num))  # (folder, file)
    return info

def download_txt_file(folder_acc, file_acc):
    url = f"https://www.sec.gov/Archives/edgar/data/{CIK}/{folder_acc}/{file_acc}.txt"
    res = requests.get(url, headers=HEADERS)
    if res.status_code == 200:
        file_path = os.path.join(SAVE_FOLDER, f"{file_acc}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(res.text)
        print(f"Downloaded: {file_path}")
    else:
        print(f"Failed to download: {file_acc} (status code: {res.status_code})")

# === MAIN ===
filings = get_10k_filing_info(CIK)

for folder_acc, file_acc in filings:
    try:
        download_txt_file(folder_acc, file_acc)
        time.sleep(0.5)
    except Exception as e:
        print(f"Error downloading {file_acc}: {e}")


Downloaded: apple_10k_txts/0000320193-24-000123.txt
Downloaded: apple_10k_txts/0000320193-23-000106.txt
Downloaded: apple_10k_txts/0000320193-22-000108.txt
Downloaded: apple_10k_txts/0000320193-21-000105.txt
Downloaded: apple_10k_txts/0000320193-20-000096.txt
Downloaded: apple_10k_txts/0000320193-19-000119.txt
Downloaded: apple_10k_txts/0000320193-18-000145.txt
Downloaded: apple_10k_txts/0000320193-17-000070.txt
Downloaded: apple_10k_txts/0001628280-16-020309.txt
Downloaded: apple_10k_txts/0001193125-15-356351.txt
Downloaded: apple_10k_txts/0001193125-14-383437.txt


In [8]:
with open("apple_10k_txts/0000320193-17-000070.txt", "r", encoding="utf-8") as f:
    content = f.read()

content[:1000]

'<SEC-DOCUMENT>0000320193-17-000070.txt : 20171103\n<SEC-HEADER>0000320193-17-000070.hdr.sgml : 20171103\n<ACCEPTANCE-DATETIME>20171103080137\nACCESSION NUMBER:\t\t0000320193-17-000070\nCONFORMED SUBMISSION TYPE:\t10-K\nPUBLIC DOCUMENT COUNT:\t\t97\nCONFORMED PERIOD OF REPORT:\t20170930\nFILED AS OF DATE:\t\t20171103\nDATE AS OF CHANGE:\t\t20171103\n\nFILER:\n\n\tCOMPANY DATA:\t\n\t\tCOMPANY CONFORMED NAME:\t\t\tAPPLE INC\n\t\tCENTRAL INDEX KEY:\t\t\t0000320193\n\t\tSTANDARD INDUSTRIAL CLASSIFICATION:\tELECTRONIC COMPUTERS [3571]\n\t\tIRS NUMBER:\t\t\t\t942404110\n\t\tSTATE OF INCORPORATION:\t\t\tCA\n\t\tFISCAL YEAR END:\t\t\t0930\n\n\tFILING VALUES:\n\t\tFORM TYPE:\t\t10-K\n\t\tSEC ACT:\t\t1934 Act\n\t\tSEC FILE NUMBER:\t001-36743\n\t\tFILM NUMBER:\t\t171174673\n\n\tBUSINESS ADDRESS:\t\n\t\tSTREET 1:\t\tONE INFINITE LOOP\n\t\tCITY:\t\t\tCUPERTINO\n\t\tSTATE:\t\t\tCA\n\t\tZIP:\t\t\t95014\n\t\tBUSINESS PHONE:\t\t(408) 996-1010\n\n\tMAIL ADDRESS:\t\n\t\tSTREET 1:\t\tONE INFINITE LOOP\n\t

In [40]:
import re


# Regex to find all ITEM headings and their positions
pattern = re.compile(r'(ITEM\s+\d+[A]?(?:\.\s*[^\\n]*)?)', re.IGNORECASE)
pattern = re.compile(r'ITEM\s+\d+[A]?', re.IGNORECASE)
matches = list(pattern.finditer(content))

# Create a dictionary of sections
sections = {}
for i in range(len(matches)):
    section_title = matches[i].group().strip().upper()
    start_index = matches[i].start()
    end_index = matches[i+1].start() if i+1 < len(matches) else len(content)
    sections[section_title] = content[start_index:end_index].strip()

sections['ITEM 1A']

'Item 1A. </font></div></td><td style="vertical-align:top;"><div style="line-height:120%;text-align:justify;font-size:9pt;"><font style="font-family:Helvetica,sans-serif;font-size:9pt;font-weight:bold;">Risk Factors</font></div></td></tr></table><div style="line-height:120%;padding-top:8px;text-align:justify;font-size:9pt;"><font style="font-family:Helvetica,sans-serif;font-size:9pt;">The following discussion of risk factors contains forward-looking statements. These risk factors may be important to understanding other statements in this Form 10-K. The following information should be read in conjunction with Part II, Item&#160;7, &#8220;Management&#8217;s Discussion and Analysis of Financial Condition and Results of Operations&#8221; and the consolidated financial statements and related notes in Part II, Item&#160;8, &#8220;Financial Statements and Supplementary Data&#8221; of this Form 10-K.</font></div><div style="line-height:120%;padding-top:16px;text-align:justify;font-size:9pt;"><

In [52]:
sections['ITEM 14']

'Item 14. </font></div></td><td style="vertical-align:top;"><div style="line-height:120%;text-align:justify;font-size:9pt;"><font style="font-family:Helvetica,sans-serif;font-size:9pt;font-weight:bold;">Principal Accounting Fees and Services</font></div></td></tr></table><div style="line-height:120%;padding-top:8px;text-align:justify;font-size:9pt;"><font style="font-family:Helvetica,sans-serif;font-size:9pt;">The information required by this Item is set forth under the subheadings &#8220;Fees Paid to Auditors&#8221; and &#8220;Policy on Audit Committee Pre-Approval of Audit and Non-Audit Services Performed by the Independent Registered Public Accounting Firm&#8221; under the proposal &#8220;Ratification of Appointment of Independent Registered Public Accounting Firm&#8221; in the Company&#8217;s </font><font style="font-family:Helvetica,sans-serif;font-size:9pt;">2018</font><font style="font-family:Helvetica,sans-serif;font-size:9pt;">&#32;Proxy Statement to be filed with the SEC with

In [54]:
import html 

# Remove HTML tags using regex
cleaned_text = re.sub(r'<[^>]*>', '', sections['ITEM 15'])
cleaned_text = html.unescape(cleaned_text)

# Print cleaned text
print(cleaned_text)

Item 15.Exhibits, Financial Statement Schedules(a)Documents filed as part of this report(1)All financial statementsIndex to Consolidated Financial Statements PageConsolidated Statements of Operations for the years ended September 30, 2017, September 24, 2016 and September 26, 2015 39Consolidated Statements of Comprehensive Income for the years ended September 30, 2017, September 24, 2016 and September 26, 2015 40Consolidated Balance Sheets as of September 30, 2017 and September 24, 2016 41Consolidated Statements of Shareholders’ Equity for the years ended September 30, 2017, September 24, 2016 and September 26, 2015 42Consolidated Statements of Cash Flows for the years ended September 30, 2017, September 24, 2016 and September 26, 2015 43Notes to Consolidated Financial Statements 44Selected Quarterly Financial Information (Unaudited) 68Reports of Ernst & Young LLP, Independent Registered Public Accounting Firm 70(2)Financial Statement SchedulesAll financial statement schedules have bee

In [39]:
len(sections)

20

In [49]:
for key,value in sections.items():
    print(key)

ITEM 1
ITEM 1A
ITEM 2
ITEM 3
ITEM 4
ITEM 5
ITEM 6
ITEM 7
ITEM 7A
ITEM 8
ITEM 9
ITEM 9A
ITEM 10
ITEM 11
ITEM 12
ITEM 13
ITEM 14
ITEM 15
ITEM 16
ITEM 601
